# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_df = pd.read_csv('cities.csv', index_col='City_ID')
city_df.head()

,country_code,city,lat,lng,date,max_temp,humidity,wind_speed,cloudiness
City_ID,,,,,,,,,
0,cl,punta arenas,-76.361239,-99.479026,1651076968,-27.51,100,100,68
1,ru,sentyabrskiy,40.063484,156.716326,1651076969,13.50,99,99,100
2,ph,siocon,7.679821,122.162128,1651076969,25.03,94,94,100
3,ca,yellowknife,73.770045,-112.568548,1651076969,-10.42,90,90,100
4,cn,pingliang,34.891101,106.799152,1651076969,8.51,90,90,100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#Create a map using state centroid coordiantes to set markers 
locations = city_df[['lat','lng']]
w = city_df['humidity'].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=w,
                            dissipating=False, max_intensity=10,
                            point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [61]:
#set weather criteria parameter
max_cloud = 20
max_humidity = 20
max_windspeed = 25
max_temp = 25

#store into new dataframe
hotel_df = city_df.loc[(city_df['cloudiness'] <= max_cloud) & (city_df['humidity'] <= max_humidity) & (city_df['wind_speed'] <= max_windspeed)& (city_df['max_temp'] <= max_temp)].reset_index()

hotel_df.reset_index(inplace=True)

hotel_df

,index,City_ID,country_code,city,lat,lng,date,max_temp,humidity,wind_speed,cloudiness,Hotel Name
0,0,247,ar,san rafael,-34.383029,-67.367454,1651077009,19.12,18,18,1,NaN
1,1,284,cl,diego de almagro,-26.740405,-69.615504,1651077017,11.51,10,10,0,NaN
2,2,801,za,calvinia,-30.724498,19.112890,1651077115,21.29,20,20,0,NaN
3,3,850,cl,vicuna,-29.470234,-69.125486,1651077124,12.94,5,5,0,NaN
4,4,961,us,payson,34.535390,-110.903977,1651077143,18.93,12,12,0,NaN
5,5,1047,us,flagstaff,35.717198,-112.006820,1651077158,18.42,11,11,19,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    #"keyword": "hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address["results"][0]["name"], indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,index,City_ID,country_code,city,lat,lng,date,max_temp,humidity,wind_speed,cloudiness,Hotel Name
0,0,247,ar,san rafael,-34.383029,-67.367454,1651077009,19.12,18,18,1,Puesto El Plumero
1,1,284,cl,diego de almagro,-26.740405,-69.615504,1651077017,11.51,10,10,0,NaN
2,2,801,za,calvinia,-30.724498,19.112890,1651077115,21.29,20,20,0,Rock Ridge Manor
3,3,850,cl,vicuna,-29.470234,-69.125486,1651077124,12.94,5,5,0,Cerro Urcuschun
4,4,961,us,payson,34.535390,-110.903977,1651077143,18.93,12,12,0,Woods Canyon Lake Group Campground
5,5,1047,us,flagstaff,35.717198,-112.006820,1651077158,18.42,11,11,19,The Grand Hotel at the Grand Canyon


In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country_code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_1 = hotel_df[["lat", "lng"]]

In [75]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations_1,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

['\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>renqiu</dd>\n<dt>Country</dt><dd>cn</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>hengshui</dd>\n<dt>Country</dt><dd>cn</dd>\n</dl>\n']
